# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [89]:
# import libraries
import sys
from sqlalchemy import create_engine
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])
import sqlite3
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import pickle

import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [90]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
engine.table_names()
df = pd.read_sql_table('df', con=engine)
# extract values from X and y
X = df['message'].values
Y = df.iloc[:,4:]
category_names = Y.columns
df

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,14,Information about the National Palace-,Informtion au nivaux palais nationl,direct,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,15,Storm at sacred heart of jesus,Cyclone Coeur sacr de jesus,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,16,"Please, we need tents and water. We are in Sil...",Tanpri nou bezwen tant avek dlo nou zon silo m...,direct,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,17,"I would like to receive the messages, thank you",Mwen ta renmen jouin messag yo. Merci,direct,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,18,I am in Croix-des-Bouquets. We have health iss...,"Nou kwadebouke, nou gen pwoblem sant m yo nan ...",direct,1.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


### 2. Write a tokenization function to process your text data

In [91]:
def tokenize(text):
    """
    Tokenize message data
    Parameters:
    text: string
    
    Returns:
    clean_tokens: List of tokens
    """
    
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [92]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
       ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [93]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [94]:
def main():
    test_pred = pipeline.predict(X_test)
    for i in range(len(category_names)): 
        print(category_names[i]) 
        print(classification_report(Y_test[category_names[i]], test_pred[:, i]))

main()

related
             precision    recall  f1-score   support

        0.0       0.62      0.40      0.49      1567
        1.0       0.83      0.92      0.87      4987

avg / total       0.78      0.80      0.78      6554

request
             precision    recall  f1-score   support

        0.0       0.91      0.96      0.94      5452
        1.0       0.75      0.55      0.64      1102

avg / total       0.89      0.89      0.89      6554

offer
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      6523
        1.0       0.25      0.06      0.10        31

avg / total       0.99      0.99      0.99      6554

aid_related
             precision    recall  f1-score   support

        0.0       0.75      0.86      0.80      3843
        1.0       0.75      0.59      0.66      2711

avg / total       0.75      0.75      0.74      6554

medical_help
             precision    recall  f1-score   support

        0.0       0.94      0.98      0

### 6. Improve your model
Use grid search to find better parameters. 

In [97]:
# Improved parameters 
parameters = {
        'clf__estimator__n_estimators': [10]
    }
    # new model with improved parameters
cv = GridSearchCV(estimator=pipeline, param_grid=parameters, cv=None, verbose=12, n_jobs=-1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [98]:
def main():
    test_pred = pipeline.predict(X_test)
    for i in range(len(category_names)): 
        print(category_names[i]) 
        print(classification_report(Y_test[category_names[i]], test_pred[:, i]))

main()

related
             precision    recall  f1-score   support

        0.0       0.62      0.40      0.49      1567
        1.0       0.83      0.92      0.87      4987

avg / total       0.78      0.80      0.78      6554

request
             precision    recall  f1-score   support

        0.0       0.91      0.96      0.94      5452
        1.0       0.75      0.55      0.64      1102

avg / total       0.89      0.89      0.89      6554

offer
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      6523
        1.0       0.25      0.06      0.10        31

avg / total       0.99      0.99      0.99      6554

aid_related
             precision    recall  f1-score   support

        0.0       0.75      0.86      0.80      3843
        1.0       0.75      0.59      0.66      2711

avg / total       0.75      0.75      0.74      6554

medical_help
             precision    recall  f1-score   support

        0.0       0.94      0.98      0

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [99]:
model = main()

related
             precision    recall  f1-score   support

        0.0       0.62      0.40      0.49      1567
        1.0       0.83      0.92      0.87      4987

avg / total       0.78      0.80      0.78      6554

request
             precision    recall  f1-score   support

        0.0       0.91      0.96      0.94      5452
        1.0       0.75      0.55      0.64      1102

avg / total       0.89      0.89      0.89      6554

offer
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      6523
        1.0       0.25      0.06      0.10        31

avg / total       0.99      0.99      0.99      6554

aid_related
             precision    recall  f1-score   support

        0.0       0.75      0.86      0.80      3843
        1.0       0.75      0.59      0.66      2711

avg / total       0.75      0.75      0.74      6554

medical_help
             precision    recall  f1-score   support

        0.0       0.94      0.98      0

In [100]:
import pickle
pickle.dump(model, open('classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.